In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

In [ ]:
G = const.G.value
M_SUN = const.M_sun.value
M_EARTH = const.M_earth.value
M_MOON = 7.34767309e22

R_SUN = const.R_sun.value
R_EARTH = const.R_earth.value
R_MOON = 1737400.0

AU = const.au.value

a_Earth = 1.000001018 * AU
e_Earth = 0.0167086

a_Moon = 384748000.0
e_Moon = 0.0549006

DAY = 24 * 3600
MONTH = 27.321661 * DAY
YEAR = 365.25636 * DAY

MASSES = np.array([M_SUN, M_EARTH, M_MOON])
T_FINAL = 1.0 * MONTH
DT = 300
t_array = np.arange(0, T_FINAL, DT)

In [ ]:
def solve_kepler_w_newton_backtrack_bisection(M, e, tol=1e-12, max_iter=50):
    M = np.mod(M, 2*np.pi)
    
    if e < 0.8:
        E = M
    else:
        E = np.pi
    
    for i in range(1, max_iter + 1):
        f = E - e*np.sin(E) - M
        fp = 1 - e*np.cos(E)
        
        if abs(f) < tol:
            return E, i
        
        step = -f / fp
        E_new = E + step
        
        lmbda = 1.0
        while abs(E_new - E) > np.pi:
            lmbda *= 0.5
            E_new = E + lmbda*step
            if lmbda < 1e-6:
                break
        
        if abs(E_new - E) > np.pi:
            E_new = 0.5 * (E + M)
        
        E = E_new
    
    return E, max_iter

In [ ]:
def get_n_body_acceleration(positions, masses):
    N = positions.shape[0]
    acc = np.zeros_like(positions)

    for i in range(N):
        for j in range(N):
            if i != j:
                r_vec = positions[j] - positions[i]
                r_dist = np.linalg.norm(r_vec)
                
                acc[i] += G * masses[j] * r_vec / (r_dist**3)
    return acc

In [ ]:
def velocity_verlet_simulation(r0, v0, masses, t_array):
    dt = t_array[1] - t_array[0]
    steps = len(t_array)
    N = len(masses)

    r_vals = np.zeros((steps, N, 3))
    
    r = r0.copy()
    v = v0.copy()
    a = get_n_body_acceleration(r, masses)
    
    r_vals[0] = r
    
    for i in range(1, steps):
        v_half = v + 0.5 * a * dt
        
        r = r + v_half * dt
    
        a_new = get_n_body_acceleration(r, masses)

        v = v_half + 0.5 * a_new * dt

        a = a_new

        r_vals[i] = r
    return r_vals

In [ ]:
def get_kepler_state(a, e, M_cent, t, M0=0):
    T = 2 * np.pi * np.sqrt(a**3 / G*M_cent)
    M = M0 + 2 * np.pi * (t / T)
    E, _ = solve_kepler_w_newton_backtrack_bisection(M, e)
    
    r_x = a * (np.cos(E) - e) # equals a*cos(E) - a*e
    r_y = a * np.sqrt(1 - e**2) * np.sin(E)
    
    n = np.sqrt(G*M_cent / a**3)
    v_x = -a * n * np.sin(E) / (1 - e * np.cos(E))
    v_y = a * n * np.sqrt(1 - e**2) * np.cos(E) / (1 - e * np.cos(E))
    
    r = np.array([r_x, r_y, 0.0])
    v = np.array([v_x, v_y, 0.0])
    
    return r, v

In [ ]:
r_earth_0, v_earth_0 = get_kepler_state(a_Earth, e_Earth, M_SUN, 0)
r_moon_rel_0, v_moon_rel_0 = get_kepler_state(a_Moon, e_Moon, M_EARTH, 0)

r0 = np.array([
    [0.0, 0.0, 0.0],
    r_earth_0,
    r_earth_0 + r_moon_rel_0
])

v0 = np.array([
    [0.0, 0.0, 0.0],
    v_earth_0,
    v_earth_0 + v_moon_rel_0
])

v_initial = v0[2].copy()

T_final = t_array[-1]
r_earth_final_newton, _ = get_kepler_state(a_Earth, e_Earth, M_EARTH, T_final)
r_moon_rel_final_newton, _ = get_kepler_state(a_Moon, e_Moon, M_MOON, T_final)
TARGET_POS_ABS = r_earth_final_newton + r_moon_rel_final_newton

In [ ]:
def calculate_loss(v_moon_guess):
    curr_v0 = v0.copy()
    curr_v0[2] = v_moon_guess
    history = velocity_verlet_simulation(r0, curr_v0, MASSES, t_array)
    final_pos_moon = history[-1, 2, :]
    return np.linalg.norm(final_pos_moon - TARGET_POS_ABS)

In [ ]:
def df(v_current):
    h = 1e-5
    grad = np.zeros(3)
    base_loss = calculate_loss(v_current)
    for i in range(3):
        v_shifted = v_current.copy()
        v_shifted[i] += h
        grad[i] = (calculate_loss(v_shifted) - base_loss) / h
    return grad

In [ ]:
def gradient_nd(df, x0, calculate_loss, precision=1e-5, gamma=0.01, niter=100):
    cur_x = np.array(x0, dtype=float)
    pos_history = [np.copy(cur_x)]
    prev_df = df(cur_x)

    step_size = precision * 2.0
    iters = 0

    while step_size > precision and iters < niter:
        prev_x = np.copy(cur_x)
        cur_x -= gamma * prev_df
        cur_df = df(cur_x)
        
        step_size = np.sqrt(np.dot(cur_x - prev_x, cur_x - prev_x))
        den = np.dot(cur_df - prev_df,  cur_df - prev_df)
        
        if den == 0:
            break

        num = np.abs(np.dot(cur_x - prev_x,  cur_df - prev_df))
        gamma = num / den

        prev_df = cur_df
        pos_history.append(np.copy(cur_x))
        iters += 1
        
    return iters, cur_x, pos_history